# OpenFermion and XACC
Here we demonstrate how to integrate data structures from OpenFermion with XACC, and specifically the XACC-VQE application. XACC and XACC-VQE are both written in C++, but also expose a Python API that enables quick scripting. The VQE application now exposes functions that provide interoperability with OpenFermion - Hamiltonians may be developed and described in Python and OpenFermion, but run on any available XACC Accelerator, and specifically QPUs from Rigetti and IBM. 

Here we demonstrate IBM integration by programming the deuteron 2x2 Hamiltonian as a FermionOperator, and computing its energy on the IBMQX5 chip.

## Docker instructions
```bash
$ git clone https://github.com/ornl-qci/xacc-vqe && cd xacc-vqe/examples/openfermion
$ docker run --name xacc-notebook -it -p 8888:8888 -d -v $(pwd):/home/notebooks mccaskey/xacc-all-gate-jupyter-fc26
$ docker logs xacc-notebook (to see Jupyter URL)
```
Copy and paste the Jupyter URL into your browser, and open this Jupyter notebook.

Enter the docker container
```bash
$ docker exec -it xacc-notebook bash
```

In the container, create a .ibm_config file in your $HOME directory. Add the following
```bash
url: https://quantumexperience.ng.bluemix.net
key: YOUR_API_KEY
```

## Programming 2x2 Deuteron Hamiltonian with OpenFermion (Simulation)

In [1]:
import numpy as np
from openfermion.ops import FermionOperator
import sys
sys.path.append('/usr/local/xacc/lib/python')
import pyxaccvqe as vqe
import pyxacc as xacc
from pyxacc import InstructionParameter

# Matrix elements
mat2=np.array([[ -0.43658111,  -4.28660705],
               [ -4.28660705,  12.25      ]])

# Create the Hamiltonian with OpenFermion
H_f = FermionOperator()              
for i,j in [(0,0),(0,1),(1,0),(1,1)]:
    H_f += FermionOperator('{}^ {}'.format(i,j), mat2[i,j])

print('Jordan-Wigner: ', vqe.compile(H_f))
print('Expected E = ', vqe.execute(H_f).energy)

# Create the Ansatz Circuit for our VQE run
statePrep = xacc.gate.GateFunction('ansatz', [InstructionParameter('theta')])
statePrep.add(xacc.gate.create('X',[0]))
statePrep.add(xacc.gate.create('Ry',[1],[InstructionParameter('theta')]))
statePrep.add(xacc.gate.create('CNOT',[1,0]))

# Specify the TNQVM Simulator
accelerator = 'tnqvm'

# Execute, here we are just going to compute the energy 
# at the optimal angle given by theory. 
tnqvmResult = vqe.execute(H_f, **{'task':'vqe', 'ansatz':statePrep})
print('E = ', tnqvmResult.angles, tnqvmResult.energy)

/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Jordan-Wigner:  (-2.1433,0) X0 X1 + (-2.1433,0) Y0 Y1 + (5.90671,0) + (-6.125,0) Z1 + (0.21829,0) Z0
Expected E =  -1.7491614275620793
E =  [0.59423881] -1.7491609673685549


## Run on IBM Simulator

In [ ]:
# Switch to IBM Accelerator
accelerator = 'ibm'

# By default we can run on the IBM simulator
# and we want to max out the number of shots
xacc.setOption('ibm-shots', '8192')

ibmqx5Result = vqe.execute(H_f, **{
    'task':'compute-energy',
    'vqe-params':str(tnqvmResult.angles[0]), 'ansatz':statePrep,
    'accelerator':accelerator})
print('E = ', ibmqx5Result.energy)

Job Response: RUNNING, queue: EXECUTINGN_QUEUE position 3E =  -0.4425726589355472
Job Response: RUNNING, queue: PENDING_IN_QUEUE position 3

## Run on IBMQX5

In [ ]:
# Switch to IBMQX5
xacc.setOption('ibm-backend','ibmqx5') 

# Execute, here, for time, we are just going to compute the energy 
# at the optimal angle given by theory. Furthermore, 
# XACC provides support for various error-mitigation strategies
# Here we tell the execution to correct itself for qubit readout errors
# And we run on qubits 10 and 9, due to its low CNOT error rate
ibmqx5Result = vqe.execute(H_f, **{
    'task':'compute-energy',
    'vqe-params':str(tnqvmResult.angles[0]), 'ansatz':statePrep,
    'accelerator':accelerator, 'qubit-map':[10,9], 
    'error-mitigation':['correct-readout-errors']
    })
print('E = ', ibmqx5Result.energy)